In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as plt_patches
import matplotlib.collections as plt_collections

For any convex polygon (drawn with thick lines) given as a list of its corners:

[ (x1, y1), (x2, y2) , … ] 

find a polygon that is enlarged as shown in the the diagram.

Every distance between corresponding sides of the given and the enlarged polygons equals one.

In [ ]:
def render(polygons, additional = False):
    _, ax = plt.subplots()
    patches = []
    for polygon in polygons:
        p = plt_patches.Polygon(np.array(polygon), closed=True)
        patches.append(p)
        x, y = zip(*polygon)
        plt.scatter(x, y)
    pc = plt_collections.PatchCollection(patches, alpha=0.5)
    ax.add_collection(pc)
    ax.autoscale()
    if additional:
        additional(ax)
    plt.axis('equal')
    plt.show()


In [ ]:
w = 1
polygon = [
    np.array([3, 2]),
    np.array([2, 4.5]),
    np.array([4.5, 6]),
    np.array([7, 4.5]),
    np.array([6, 2]),
]
render([polygon])


In [ ]:
def direction_vector(v1, v2):
    # ab = b - a
    return v2 - v1

In [ ]:
def unit_vector(vector):
    # a / |a|
    return vector / np.linalg.norm(vector)

In [ ]:
def get_angle(v1, v2, counterclockwise=False):
    # cos^-1 [ (a,b) / (|a| |b|)].
    a = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    if counterclockwise:
        a = (2 * np.pi) - a
    return a / 2


In [ ]:
def S(angle):
    return 1 / np.sin(angle)

In [ ]:
def R(angle):
    return np.array([[np.cos(angle), -np.sin(angle)],
                     [np.sin(angle), np.cos(angle)]])


In [ ]:
def find_neighbors(p):
    r = []
    l = len(p)
    for i in range(l):
        if (i == 0):
            r.append([p[i], p[l-1], p[i+1]])
        elif (i == l - 1):
            r.append([p[i], p[i-1], p[0]])
        else:
            r.append([p[i], p[i-1], p[i+1]])
    return r


In [ ]:
vectors = find_neighbors(polygon)
vectors[0]

In [ ]:
scaled_polygon = []
for vector, prev, next in vectors:
    s1 = direction_vector(vector, prev)
    s1 = unit_vector(s1)
    s2 = direction_vector(vector, next)
    s2 = unit_vector(s2)
    
    angle = get_angle(s1, s2, True)
    
    rotated_direction_vector = R(angle).dot(s2)
    scaled_direction_vector = S(angle) * rotated_direction_vector
    moved_direction_vector =  scaled_direction_vector + vector
    
    scaled_polygon.append(moved_direction_vector)

render([polygon, scaled_polygon])


Single vector example

In [ ]:
vector = polygon[0]
prev = polygon[len(polygon) - 1]
next = polygon[1]

def show(ax):
    ax.scatter(*vector, color='red')
    ax.annotate('vector', (vector))
    ax.scatter(*prev, color='green')
    ax.annotate('prev', (prev))
    ax.scatter(*next, color='green')
    ax.annotate('next', (next))

render([polygon], show)

In [ ]:
s1 = direction_vector(vector, prev)
s1 = unit_vector(s1)
s2 = direction_vector(vector, next)
s2 = unit_vector(s2)

def show(ax):
    ax.scatter(*vector, color='red')
    ax.annotate('vector', (vector))
    ax.arrow(*vector, *(s1 - vector), width=0.03, color='blue')
    ax.scatter(*s1, color='blue', alpha=0.7)
    ax.annotate('s1 unit vector', (s1))
    ax.arrow(*vector, *(s2 - vector), width=0.03, color='orange')
    ax.scatter(*s2, color='orange', alpha=0.7)
    ax.annotate('s2 unit vector', (s2))

render([polygon], show)


In [ ]:
angle = get_angle(s1, s2, True)
rotated_direction_vector = R(angle).dot(s2)

def show(ax):
    ax.scatter(*vector, color='red')
    ax.annotate('vector', (vector))
    ax.scatter(*s2, color='blue')
    # rotation example
    for a in np.linspace(0, angle, 10):
        ax.scatter(*R(a).dot(s2), color='grey', alpha=0.5)
    #
    ax.scatter(*rotated_direction_vector, color='green')
    ax.annotate('rotaded s1 unit vector', (rotated_direction_vector))

render([polygon], show)


In [ ]:
scaled_rotated_vector = S(angle) * rotated_direction_vector
moved_direction_vector = scaled_rotated_vector + vector

def show(ax):
    ax.scatter(*vector, color='red')
    ax.annotate('vector', (vector))
    ax.scatter(*rotated_direction_vector, color='pink')
    ax.annotate('rotaded vector', (rotated_direction_vector))
    ax.scatter(*scaled_rotated_vector, color='green')
    ax.annotate('scaled vector', (scaled_rotated_vector))
    ax.arrow(*rotated_direction_vector, *
             (scaled_rotated_vector - rotated_direction_vector), width=0.03 , color='pink')

    ax.scatter(*moved_direction_vector, color='green')
    ax.annotate('vector in new position', (moved_direction_vector))
    ax.arrow(*scaled_rotated_vector, *
             (moved_direction_vector - scaled_rotated_vector), width=0.03, color='green')

render([polygon, scaled_polygon], show)
